In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.python.keras.callbacks import TensorBoard
from time import time
import matplotlib.pyplot as plt
from scipy import signal

In [2]:
full_df = pd.read_csv("../resources/full_dataset.csv")

In [3]:
def generate_spectrogram(full_df, spec_shape, fs=200, window=('tukey', 0.25), nperseg=50, noverlap=None):
    full_df = full_df.sample(frac=1).reset_index(drop=True)
    labels = full_df["labels"]
    features = full_df.drop(labels="labels", axis='columns')
    features = features.values.reshape(-1, 400, 4)
    arr = np.zeros((1000, spec_shape[0], spec_shape[1], 4))
    first = True
    for i in range(1000):
        _, _, spec_1 = signal.spectrogram(features[i][:, 0], fs=fs, window=window,
                                               nperseg=nperseg, noverlap=noverlap)
        spec_1 = spec_1.reshape(spec_shape[0], spec_shape[1], 1)
        _, _, spec_2 = signal.spectrogram(features[i][:, 1], fs=fs, window=window,
                                               nperseg=nperseg, noverlap=noverlap)
        spec_2 = spec_2.reshape(spec_shape[0], spec_shape[1], 1)
        _, _, spec_3 = signal.spectrogram(features[i][:, 2], fs=fs, window=window,
                                               nperseg=nperseg, noverlap=noverlap)
        spec_3 = spec_3.reshape(spec_shape[0], spec_shape[1], 1)
        _, _, spec_4 = signal.spectrogram(features[i][:, 3], fs=fs, window=window,
                                               nperseg=nperseg, noverlap=noverlap)
        spec_4 = spec_4.reshape(spec_shape[0], spec_shape[1], 1)
        if first:
            print("Spectrogram per channel shape {}".format(spec_1.shape))
            first = False
        arr[i] = np.concatenate((spec_1, spec_2, spec_3, spec_4), axis=2)
    print("arr shape {} --> {}".format(arr.shape, arr.reshape(-1, spec_shape[0] * spec_shape[1] * 4).shape))
    arr = arr.reshape(-1, spec_shape[0] * spec_shape[1] * 4)
    columns = []
    for i in range(spec_shape[0] * spec_shape[1] * 4):
        columns.append("p{}".format(i))
    new_df = pd.DataFrame(data=arr, columns=columns)
    new_df["labels"] = labels
    return (new_df)

In [4]:
data = generate_spectrogram(full_df, spec_shape=(26, 8))

Spectrogram per channel shape (26, 8, 1)
arr shape (1000, 26, 8, 4) --> (1000, 832)


In [5]:
def preprocess_data(data, spec_shape):
    labels = data["labels"].values
    labels = np.where(labels == "Lights-on", 1, 0) # 1 for Lights-on and 0 for Turn-off
    labels = tf.keras.utils.to_categorical(labels)
    data = data.drop(labels="labels", axis='columns')
    data = data.values.reshape(-1, spec_shape[0], spec_shape[1], 4)
    return (data, labels)

In [6]:
test = data.head(100)
training = data.tail(900)

In [7]:
features, labels = preprocess_data(training, spec_shape=(26, 8))

In [8]:
tensorboard = TensorBoard(log_dir="logs/model_1_{}".format(time()))

model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, strides=(1, 1), padding="same",
                                 activation='relu', kernel_initializer='he_normal', input_shape=(26, 8, 4)))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, strides=(1, 1),
                                 padding="same", activation='relu', kernel_initializer='he_normal'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding="same"))
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, strides=(1, 1),
                                 padding="same", activation='relu', kernel_initializer='he_normal'))
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, strides=(1, 1),
                                 padding="same", activation='relu', kernel_initializer='he_normal'))
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, strides=(1, 1),
                                 padding="same", activation='relu', kernel_initializer='he_normal'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding="same"))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024, activation='relu', kernel_initializer='he_normal'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

#optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.88, nesterov=True)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 8, 64)         1088      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 8, 64)         16448     
_________________________________________________________________
batch_normalization_v2 (Batc (None, 26, 8, 64)         256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 4, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 4, 128)        32896     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 4, 128)        65664     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 4, 128)        6

In [9]:
model.fit(x=features,
          y=labels,
          batch_size=24,
          epochs=10,
          validation_split=0.15,
          callbacks=[tensorboard])

Train on 765 samples, validate on 135 samples
Epoch 1/10
765/765 [==============================] - 2s 3ms/sample - loss: 1.2557 - accuracy: 0.6693 - val_loss: 0.9674 - val_accuracy: 0.6074
Epoch 2/10
765/765 [==============================] - 2s 2ms/sample - loss: 0.4962 - accuracy: 0.8288 - val_loss: 0.5255 - val_accuracy: 0.7556
Epoch 3/10
765/765 [==============================] - 2s 2ms/sample - loss: 0.3088 - accuracy: 0.8915 - val_loss: 0.5347 - val_accuracy: 0.7704
Epoch 4/10
765/765 [==============================] - 2s 2ms/sample - loss: 0.1735 - accuracy: 0.9346 - val_loss: 0.3241 - val_accuracy: 0.8370
Epoch 5/10
765/765 [==============================] - 2s 2ms/sample - loss: 0.1001 - accuracy: 0.9647 - val_loss: 0.1214 - val_accuracy: 0.9481
Epoch 6/10
765/765 [==============================] - 2s 2ms/sample - loss: 0.0746 - accuracy: 0.9699 - val_loss: 0.0748 - val_accuracy: 0.9704
Epoch 7/10
765/765 [==============================] - 2s 2ms/sample - loss: 0.0680 - accur

In [10]:
k

NameError: name 'k' is not defined

In [ ]:
test_features, test_labels = preprocess_data(test, spec_shape=(26, 8))

In [ ]:
metrics = model.evaluate(x=test_features, y=test_labels)

In [ ]:
metrics